<img src="https://mediapipe.dev/images/mobile/hand_landmarks.png" width=1200>

In [1]:
import cv2 as cv
import mediapipe
import numpy as np
import pyautogui
import os
os.chdir('../../data')

In [2]:
mphands = mediapipe.solutions.hands
detector = mphands.Hands()
draw = mediapipe.solutions.drawing_utils

wscreen,  hscreen = pyautogui.size()
px, py = 0, 0
cx, cy = 0, 0

In [3]:
def handLandmarks(img, frame):
    landmarkList = []
    landmarkPositions = detector.process(img)

    landmarkCheck = landmarkPositions.multi_hand_landmarks

    if landmarkCheck:
        for hand in landmarkCheck:
            for i, landmark in enumerate(hand.landmark):
                draw.draw_landmarks(frame, hand, mphands.HAND_CONNECTIONS)
                h, w, c = img.shape
                centerX, centerY = int(landmark.x*w), int(landmark.y*h)
                landmarkList.append([i, centerX, centerY])
    return landmarkList, frame

In [4]:
def fingers(landmarks):
    fingerTips = []
    tipIds = [4, 8, 12, 16, 20]

    if landmarks[4][1] > landmarks[3][1]:
        fingerTips.append(1)
    else:
        fingerTips.append(0)

    for i in range(1, 5):
        if landmarks[tipIds[i]][2] < landmarks[tipIds[i] - 3][2]:
            fingerTips.append(1)
        else:
            fingerTips.append(0)
    return fingerTips

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [ ]:
cap = cv.VideoCapture(1)

while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        rgbFrame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        lmList, frame = handLandmarks(rgbFrame, frame)

        if len(lmList) !=0:
            x1, y1 = lmList[8][1:]
            x2, y2 = lmList[12][1:]
            finger = fingers(lmList)

            if finger[1] == 1 and finger[2] == 0:
                x3 = np.interp(x1, (0, int(cap.get(cv.CAP_PROP_FRAME_WIDTH))), (0, wscreen))
                y3 = np.interp(y1, (0, int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))), (0, hscreen))

                cx = px + (x3 - px)/6
                cy = py + (y3 - py)/6

                pyautogui.moveTo(wscreen-cx, cy)
                px, py = cx, cy

            if finger[1] == 0:
                pyautogui.click()
        # cv.imshow('frame', frame)
        if cv.waitKey(1) == 27:
            break
cap.release()
cv.destroyAllWindows()